In [5]:
import chainer
import chainer.functions as F
import chainer.links as L

/Users/sanoyoh/Desktop/latest_tech/computer_vision/chainer/3env-chainer/lib/python3.6/site-packages/chainer/_environment_check.py:38: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BLAS libraries such as OpenBLAS.
For details of the issue, please see
https://docs.chainer.org/en/stable/tips.html#mnist-example-does-not-converge-in-cpu-mode-on-mac-os-x.

Also note that Chainer does not officially support Mac OS X.
Please use it at your own risk.

  ''')  # NOQA


In [6]:
class NN(chainer.Chain):
    
    # モデルの構造
    def __init__(self, n_mid_units1=10, n_mid_units2=5, n_out=3):
        super().__init__()
        with self.init_scope():
            # None →　chainer側で勝手に読み取ってくれる
            self.fc1 = L.Linear(None, n_mid_units1)
            self.fc2 = L.Linear(None, n_mid_units2)
            self.fc3 = L.Linear(None, n_out)
            # BatchNormalization
            self.bn = L.BatchNormalization(10)
            
    # 順伝播
    def __call__(self, x):
        # 変数を上書きしていく
        h = self.bn(x)
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        h = F.relu(h)
        h = self.fc3(h)
        return h

In [7]:
import numpy as np

In [8]:
np.random.seed(1)
nn = NN()
model = L.Classifier(nn)

## 学習するための設定
### Optimizerの設定

In [9]:
optimizer = chainer.optimizers.Adam() # 確率的勾配降下法

In [10]:
optimizer.setup(model)

### Iteratorの設定

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv('wine-class.csv')
t = df.iloc[:, 0]
x = df.iloc[:, 1:]

In [12]:
t = t.values -1
x = x.values

x = x. astype('f')
t = t. astype('i')

In [13]:
dataset = list(zip(x, t))
n_train = int(len(dataset)*0.7)
train, test = chainer.datasets.split_dataset_random(dataset, n_train, seed=1)

In [14]:
batchsize = 10

In [15]:
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

### Updaterの設定

In [16]:
from chainer import training

In [17]:
# device
# CPU : -1
# GPU : 0
updater = training.StandardUpdater(train_iter, optimizer, device=-1)

### TrainerとExtensionsの設定

In [18]:
from chainer.training import extensions

In [19]:
# エポックの数
epoch = 100

In [20]:
# trainerの宣言
trainer = training.Trainer(updater, (epoch, 'epoch'), out='result/wine')

# 検証データで評価
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))

trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))

# 1エポックごとに(trigger)にtrainデータに対するloss・accuracyとtestデータに対するloss・accuracy、経過時間を出力します
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))

## 学習の実行

In [21]:
trainer.run()

epoch       main/accuracy  validation/main/accuracy  main/loss   validation/main/loss  elapsed_time
1           0.338462       0.341667                  11.3522     5.08828               0.118278      
2           0.466667       0.675                     4.94452     1.68124               0.232575      
3           0.661538       0.616667                  1.22626     0.823669              0.311839      
4           0.7            0.6                       0.862125    0.857965              0.382448      
5           0.666667       0.6                       0.883349    0.857521              0.452386      
6           0.669231       0.616667                  0.883131    0.836601              0.530505      
7           0.666667       0.616667                  0.861231    0.811822              0.609342      
8           0.684615       0.616667                  0.846968    0.822331              0.681252      
9           0.641667       0.6                       0.857494    0.833789           

79          0.666667       0.616667                  0.75631     0.717537              6.54524       
80          0.683333       0.633333                  0.725883    0.720225              6.62314       
81          0.661538       0.616667                  0.723444    0.713392              6.69964       
82          0.675          0.616667                  0.732459    0.716307              6.78385       
83          0.661538       0.616667                  0.749685    0.717628              6.89718       
84          0.7            0.616667                  0.695926    0.714322              7.01797       
85          0.683333       0.616667                  0.72868     0.711305              7.09819       
86          0.669231       0.633333                  0.715179    0.722697              7.18976       
87          0.691667       0.616667                  0.705637    0.715389              7.2691        
88          0.692308       0.616667                  0.697344    0.71197          

## 結果の確認

In [22]:
import json

In [23]:
with open('result/wine/log') as f:
    logs = json.load(f)

In [24]:
results = pd.DataFrame(logs)

In [25]:
results.head()

,elapsed_time,epoch,iteration,main/accuracy,main/loss,validation/main/accuracy,validation/main/loss
0,0.118278,1,13,0.338462,11.352243,0.341667,5.088283
1,0.232575,2,25,0.466667,4.944518,0.675000,1.681242
2,0.311839,3,38,0.661538,1.226263,0.616667,0.823669
3,0.382448,4,50,0.700000,0.862125,0.600000,0.857965
4,0.452386,5,62,0.666667,0.883349,0.600000,0.857521


In [26]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# results[['main/accuracy', 'validation/main/accuracy']].plot()

## 学習済モデルを保存

In [28]:
chainer.serializers.save_npz('models/wine.npz', model)

## 学習済モデルを使用した推論(予測値の計算)

### 学習済モデルのロード

In [29]:
# モデルの構造明示
model = L.Classifier(NN())

In [31]:
chainer.serializers.load_npz('models/wine.npz', model)

### 予測値の計算
今回は一番最初のサンプルに対する予測値の計算を行う

In [32]:
x_new = x[0]
x_new.shape

(10,)

In [34]:
# 予測値の計算
# (バッチサイズ、入力変数の数)
y = model.predictor(x_new)

InvalidType: 
Invalid operation is performed in: BatchNormalization (Forward)

Expect: in_types[0].ndim >= in_types[1].ndim + 1
Actual: 1 < 2

In [35]:
x_new = x_new[np.newaxis]

In [36]:
x_new.shape

(1, 10)

In [37]:
y = model.predictor(x_new)

/Users/sanoyoh/Desktop/latest_tech/computer_vision/chainer/3env-chainer/lib/python3.6/site-packages/chainer/functions/normalization/batch_normalization.py:67: UserWarning: A batch with no more than one sample has been given to F.batch_normalization. F.batch_normalization will always output a zero tensor for such batches. This could be caused by incorrect configuration in your code (such as running evaluation while chainer.config.train=True), but could also happen in the last batch of training if non-repeating iterator is used.
  UserWarning)


In [38]:
y

variable([[ 2.3804693, -2.1527495,  1.3052889]])

In [39]:
# 上記をたして1になるようにする
y = F.softmax(y)
y

variable([[0.7396548 , 0.00794835, 0.25239697]])

In [40]:
y.array

array([[0.7396548 , 0.00794835, 0.25239697]], dtype=float32)

In [41]:
np.argmax(y.array)

0